<a href="https://colab.research.google.com/github/filanuhazelnut99/PromptDresser/blob/main/PromptDresser_%E7%92%B0%E5%A2%83%E6%B8%AC%E8%A9%A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ICCV2025] PromptDresser: Improving the Quality and Controllability of Virtual Try-On via Generative Textual Prompt and Prompt-aware Mask

https://github.com/rlawjdghek/PromptDresser


### 安裝系統所需工具

In [1]:
# 確保在起始路徑底下
%cd /content/

cmd = """
PYVERSION=3.10

echo 降級python版本到$PYVERSION
sudo apt-get update -y
sudo apt-get install -y git-lfs ffmpeg libgl1
sudo apt-get install python$PYVERSION python$PYVERSION-distutils
update-alternatives --install /usr/bin/python3 python3 /usr/bin/python$PYVERSION 1
update-alternatives --set python3 /usr/bin/python$PYVERSION
sudo apt-get install python3-pip
python3 -m pip install --upgrade pip --user
git lfs install


echo '下載專案...'
rm -r /content/PromptDresser
git clone https://github.com/rlawjdghek/PromptDresser /content/PromptDresser
cd /content/PromptDresser

echo '安裝專案所需套件...'
bash environment.txt

echo '抽換專案部分套件...'
pip install -U --force-reinstall "numpy<2" "scipy<1.12"
pip install torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0
pip install matplotlib matplotlib-inline gdown
pip install transformers==4.31 huggingface-hub==0.25.2 peft==0.4.0
pip install -U accelerate
"""

# 寫出並執行以上shell script
with open('promptdresser_startup.sh', 'w') as file:
  file.write(cmd)

!sudo bash promptdresser_startup.sh

/content
降級python版本到3.10
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,006 kB]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,310 kB]
Hit:13 https://ppa.

### Python內部：下載套件所需資源

In [2]:
import os
import gdown
import zipfile
import pathlib
from huggingface_hub import snapshot_download

In [3]:
# Directory setup
PROJECT_ROOT = pathlib.Path('/content/PromptDresser/')
PRETRAINED_ROOT = PROJECT_ROOT / 'pretrained_models'
CHECKPOINT_ROOT = PROJECT_ROOT / 'checkpoints'
TEST_FINE_ROOT = PROJECT_ROOT / 'test_fine'
TEST_COARSE_ROOT = PROJECT_ROOT / 'test_coarse'
VAE_ROOT = PRETRAINED_ROOT / 'sdxl-vae-fp16-fix'
DIRECTORY_STRUCTURE = {
  'test_coarse':{
      'image',
      'image-densepose',
      'agnostic-mask',
      'cloth',
  },
  'test_fine':{
      'image',
      'image-densepose',
      'agnostic-mask',
      'cloth',
  }
}
# Google Drive預先下載好的模型連結
CHECKPOINT_DL = 'https://drive.google.com/drive/folders/1k9CXxywG15BDm35HXCnIXd5OXkKRVq8r?usp=drive_link'

In [4]:
# 建立專案指定的路徑結構
PRETRAINED_ROOT.mkdir(parents=True, exist_ok=True)
CHECKPOINT_ROOT.mkdir(parents=True, exist_ok=True)
for key, val in DIRECTORY_STRUCTURE.items():
  if isinstance(val, set):
    for inner_key in val:
      pathlib.Path(f'{PROJECT_ROOT}/{key}/{inner_key}').mkdir(parents=True, exist_ok=True)

In [5]:
# 下載老師幫大家上傳好的checkpoint
files = gdown.download_folder(url=CHECKPOINT_DL, output=str(CHECKPOINT_ROOT))
print(files)

Retrieving folder contents


Retrieving folder 17RtdeJlV4-2BbaPYri9SFCOXjDEPjIOZ humanparsing
Processing file 1s-BJ-bVnhzrSPs9bDT__lTu5EGGc7nxn .gitkeep
Processing file 1hM9uUSbTlgeCqykMDhLdn071s2GZjxiP parsing_atr.onnx
Processing file 1e_7B2LejkXcXbxbbwci1nufYJ0O0k7gh parsing_lip.onnx
Retrieving folder 1e78fm_y3dH0OWAmBYusTJeDrMBLkXz9k paper
Processing file 10hLcHvWq2KsWAlbWbNK8mKlyYALlv-06 Latten Diffusion Model_2022.pdf
Processing file 1S2ADSN4VRGDBTpfqx_2RIT03daxIeVMq PromptDresser_paper.pdf
Retrieving folder 1Y1F7pZlvp3r1EtYDkfZHhdlFe_iTMujk VITONHD
Processing file 1zdP4ztBf0UcwTMn4LjvnwsWw9qHUTDWU args.json
Processing file 1uaK2bqOp4Bn3LP9OnqUeuH6lWyYagum_ config.yaml
Processing file 1WkH4wUzo5TWn5hQa8I9AbDqIHgR0y01h pytorch_model.bin
Processing file 1TBSEW7s0IAr2MjYWigEFYInx5AxhZTzy PromptDresser_patch.zip
Processing file 1PMsH5BWFgqfPFQeU4VQqyA3nEoTQ-vyM PromptDresser_環境測試.ipynb 的副本
Processing file 1YQpfUDKXLyCHlF5znFXqF2928kdCkFvs test.zip
Processing file 1udRKdkBVcmiMPmVTDKoKlHI8NaiICMHw Virtual_try_on_n

Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1s-BJ-bVnhzrSPs9bDT__lTu5EGGc7nxn
To: /content/PromptDresser/checkpoints/humanparsing/.gitkeep
0.00B [00:00, ?B/s]
Downloading...
From (original): https://drive.google.com/uc?id=1hM9uUSbTlgeCqykMDhLdn071s2GZjxiP
From (redirected): https://drive.google.com/uc?id=1hM9uUSbTlgeCqykMDhLdn071s2GZjxiP&confirm=t&uuid=39fd65e8-0371-42c1-96fa-da43aa9e5fbc
To: /content/PromptDresser/checkpoints/humanparsing/parsing_atr.onnx
100%|██████████| 267M/267M [00:04<00:00, 53.5MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1e_7B2LejkXcXbxbbwci1nufYJ0O0k7gh
From (redirected): https://drive.google.com/uc?id=1e_7B2LejkXcXbxbbwci1nufYJ0O0k7gh&confirm=t&uuid=55b803d2-6462-46fe-a9e8-ab1bcfa1b1e3
To: /content/PromptDresser/checkpoints/humanparsing/parsing_lip.onnx
100%|██████████| 267M/267M [00:03<00:00, 66.8MB/s]
Downloading...
From: h

['/content/PromptDresser/checkpoints/humanparsing/.gitkeep', '/content/PromptDresser/checkpoints/humanparsing/parsing_atr.onnx', '/content/PromptDresser/checkpoints/humanparsing/parsing_lip.onnx', '/content/PromptDresser/checkpoints/paper/Latten Diffusion Model_2022.pdf', '/content/PromptDresser/checkpoints/paper/PromptDresser_paper.pdf', '/content/PromptDresser/checkpoints/VITONHD/args.json', '/content/PromptDresser/checkpoints/VITONHD/config.yaml', '/content/PromptDresser/checkpoints/VITONHD/pytorch_model.bin', '/content/PromptDresser/checkpoints/PromptDresser_patch.zip', '/content/PromptDresser/checkpoints/PromptDresser_環境測試.ipynb 的副本', '/content/PromptDresser/checkpoints/test.zip', '/content/PromptDresser/checkpoints/Virtual_try_on_notes.txt']



Download completed


In [6]:
# 使用剛剛下載組合的補丁包
patch = zipfile.ZipFile(CHECKPOINT_ROOT / 'PromptDresser_patch.zip')
patch.extractall('/content/')

# 解壓縮資料集
test_ds_zip = zipfile.ZipFile(CHECKPOINT_ROOT / 'test.zip')
test_ds_zip.extractall(TEST_FINE_ROOT)

In [7]:
# 再解壓縮一份到coarse資料夾 (之後可能依據論文寫法抽換?)
test_ds_zip = zipfile.ZipFile(CHECKPOINT_ROOT / 'test.zip')
test_ds_zip.extractall(TEST_COARSE_ROOT)

In [ ]:
# 下載 SDXL base Model（Diffusers 版）
base_dir = os.path.join(PRETRAINED_ROOT, "stable-diffusion-xl-base-1.0")
snapshot_download(
    repo_id="stabilityai/stable-diffusion-xl-base-1.0",
    local_dir=base_dir,
    local_dir_use_symlinks=False  # 寫實檔，避免 symlink 在某些情況下讀不到
)

# 下載 SDXL inpainting Model（Diffusers 版）
inpaint_dir = os.path.join(PRETRAINED_ROOT, "stable-diffusion-xl-1.0-inpainting-0.1")
snapshot_download(
    repo_id="diffusers/stable-diffusion-xl-1.0-inpainting-0.1",
    local_dir=inpaint_dir,
    local_dir_use_symlinks=False
)

# 下載SDXL VAE model （官方原版）
snapshot_download(
    repo_id="madebyollin/sdxl-vae-fp16-fix",
    local_dir=VAE_ROOT,
    local_dir_use_symlinks=False
)

# 檢查必備檔案 (StableDiffusion)
needed = [
    ("base", base_dir, ["model_index.json", "scheduler/scheduler_config.json", "unet/config.json", "vae/config.json"]),
    ("inpaint", inpaint_dir, ["model_index.json", "scheduler/scheduler_config.json", "unet/config.json", "vae/config.json"]),
]
for tag, d, files in needed:
    missing = [f for f in files if not os.path.exists(os.path.join(d, f))]
    print(tag, "OK" if not missing else f"missing: {missing}")

# 檢查必備檔案 (VAE)
need = ["config.json", "diffusion_pytorch_model.safetensors"]
print("missing:", [f for f in need if not os.path.exists(os.path.join(VAE_ROOT, f))])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 57 files:   0%|          | 0/57 [00:00<?, ?it/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

LICENSE.md: 0.00B [00:00, ?B/s]

scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

unet/diffusion_flax_model.msgpack:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

unet/diffusion_pytorch_model.fp16.safete(…):   0%|          | 0.00/5.14G [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/10.3G [00:00<?, ?B/s]

unet/model.onnx:   0%|          | 0.00/7.29M [00:00<?, ?B/s]

### 執行專案提供的推論程式
(下載並整合資料集https://opendatalab.com/OpenDataLab/VITON-HD)

In [ ]:

%cd /content/PromptDresser
!ls
!CUDA_VISIBLE_DEVICES=0 python inference.py \
 --config_p "./configs/VITONHD.yaml" \
 --pretrained_unet_path "./checkpoints/VITONHD/pytorch_model.bin" \
 --skip_unpaired \
 --save_name VITONHD

# 打包測試結果的圖片並傳到掛載的Google Drive
鮪魚老師試跑幾張圖的結果: https://drive.google.com/file/d/1F9j2SHYhnydwblu7ESgk4oqQKeGOqdK0/view?usp=sharing

## Identify the directory to zip

### Subtask:
Locate the 'paired' directory within your project.


**Reasoning**:
Construct the full path to the 'paired' directory based on the project structure.



In [ ]:
paired_dir_path = PROJECT_ROOT / 'sampled_images' / 'VITONHD' / 'paired'
print(f"The path to the 'paired' directory is: {paired_dir_path}")

## Create a zip file

### Subtask:
Create a zip file of the 'paired' directory located at `/content/PromptDresser/sampled_images/VITONHD/paired`.


**Reasoning**:
Create a zip file of the 'paired' directory by iterating through its contents and adding them to the archive.



In [ ]:

output_zip_path = CHECKPOINT_ROOT / 'paired_images.zip'

with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(paired_dir_path):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, paired_dir_path)
            zipf.write(file_path, arcname)

print(f"Created zip file at: {output_zip_path}")

## Specify the destination path in google drive

### Subtask:
Determine the desired location in your mounted Google Drive where you want to save the zip file.


**Reasoning**:
Define the destination path for the zip file in the mounted Google Drive and print it.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
drive_destination_path = '/content/drive/MyDrive/PromptDresser_Paired_Images/paired_images.zip'
print(f"The zip file will be saved to: {drive_destination_path}")

## Copy the zip file to google drive

### Subtask:
Copy the created zip file to the specified Google Drive location.


**Reasoning**:
Import the shutil module, define the source and destination paths, and copy the zip file to the Google Drive destination.



In [ ]:
import shutil
import os

output_zip_path = CHECKPOINT_ROOT / 'paired_images.zip'
drive_destination_dir = '/content/drive/MyDrive/PromptDresser_Paired_Images'
drive_destination_path = os.path.join(drive_destination_dir, 'paired_images.zip')

os.makedirs(drive_destination_dir, exist_ok=True)
shutil.copyfile(output_zip_path, drive_destination_path)

print(f"Copied '{output_zip_path}' to '{drive_destination_path}'")

## Verify the upload

### Subtask:
Optionally, check if the zip file exists in the Google Drive location.


**Reasoning**:
Check if the zip file exists in the Google Drive location and print the result.



In [ ]:
if os.path.exists(drive_destination_path):
  print(f"The zip file was successfully found in Google Drive at: {drive_destination_path}")
else:
  print(f"The zip file was NOT found in Google Drive at: {drive_destination_path}")

## Summary:

### Data Analysis Key Findings

*   The 'paired' directory was successfully located at `/content/PromptDresser/sampled_images/VITONHD/paired`.
*   A zip file named `paired_images.zip` was successfully created in the `CHECKPOINT_ROOT` directory, containing the contents of the 'paired' directory with the correct relative path structure.
*   The intended destination path in Google Drive for the zip file was set to `/content/drive/MyDrive/PromptDresser_Paired_Images/paired_images.zip`.
*   The destination directory `/content/drive/MyDrive/PromptDresser_Paired_Images` was created in Google Drive.
*   The `paired_images.zip` file was successfully copied from the `CHECKPOINT_ROOT` directory to the specified Google Drive location.
*   The presence of the `paired_images.zip` file in the Google Drive location was verified.

### Insights or Next Steps

*   The successful zipping and uploading of the 'paired' directory to Google Drive provides a backup or allows for further use of this data from the cloud.
*   Consider automating this process or setting up a regular schedule if this directory is updated frequently and needs to be backed up regularly.


# 結束任務後「自殺」處理程序，以節省colab計算資源

In [ ]:
# Restart the runtime to free up resources
import os
os.kill(os.getpid(), 9)